In [0]:
processName = dbutils.widgets.get('prm_processName')
nextsourceFileDatesql= f"""select NVL(max(PROCESSD_FILE_TABLE_DATE)+1,'2023-01-01') as NEXT_SOURCE_FILE_DATE from 
pricing_analytics.processrunlogs.DELTALAKEHOUSE_PROCESS_RUNS
where PROCESS_NAME ='{processName}' and PROCESS_STATUS='Completed'"""
nextsourceFileDateDF=spark.sql(nextsourceFileDatesql)

In [0]:
from datetime import datetime


In [0]:
from datetime import datetime
dailyPricingBaseURL="https://retailpricing.blob.core.windows.net/"
dailyPricingSourceFolder="daily-pricing/"
dailypricingsourcefiledate=datetime.strptime(str(nextsourceFileDateDF.select('NEXT_SOURCE_FILE_DATE').
                                             collect()[0]['NEXT_SOURCE_FILE_DATE']),'%Y-%m-%d').strftime('%d%m%Y')
dailyPricingSourceFileName=f"PW_MW_DR_{dailypricingsourcefiledate}.csv"
dailyPricingsinkLayerName="bronze"
dailyPricingsinkstorageAccountName="mydevstoragelake"
dailyPricingsinkFolderName="daily-pricing"

In [0]:
import pandas as pd


In [0]:
dailyPricingsourceURL=dailyPricingBaseURL+dailyPricingSourceFolder+dailyPricingSourceFileName
dailyPricingPandasDF=pd.read_csv(dailyPricingsourceURL)

In [0]:
dailypricingsparkDF = spark.createDataFrame(dailyPricingPandasDF)

In [0]:
from pyspark.sql.functions import current_timestamp
dailypricingsinkFolderpath=f"abfss://{dailyPricingsinkLayerName}@{dailyPricingsinkstorageAccountName}.dfs.core.windows.net/{dailyPricingsinkFolderName}"
(
    dailypricingsparkDF
    .withColumn("source_file_load_date",current_timestamp())
    .write
    .mode("append")
    .option("header","true")
    .csv(dailypricingsinkFolderpath)
)

In [0]:
processFileDate= nextsourceFileDateDF.select('NEXT_SOURCE_FILE_DATE').collect()[0]['NEXT_SOURCE_FILE_DATE']
processstatus ="Completed"
processInsertSQL = f"""INSERT INTO pricing_analytics.processrunlogs.DELTALAKEHOUSE_PROCESS_RUNS (PROCESS_NAME,PROCESSD_FILE_TABLE_DATE,PROCESS_STATUS) VALUES('{processName}','{processFileDate}','{processstatus}') """
spark.sql(processInsertSQL)